### Build a Simple LLM Application with LCEL.

This application will translate Text from English into another language.

This is a relatively simple LLM application - it's just a single LLM call plus some prompting. 

### Open AI API Key and 
### Open Source models--Llama3(Meta), Gemma2(Google), mistral--Groq

In [32]:

import os
from dotenv import load_dotenv
load_dotenv()


True

In [33]:
import openai
openai.api_key= os.getenv("OPENAI_API_KEY")

groq_api_key= os.getenv("GROQ_API_KEY")

In [34]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI


In [35]:
model = ChatGroq(model="gemma2-9b-it", groq_api_key= groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002AEA35131A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002AEA3512840>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [36]:
from langchain_core.messages import HumanMessage, SystemMessage

messages= [
        SystemMessage(content= "Translate the following from English to French"),
        HumanMessage(content= "Hello How are you?")
]

In [37]:
result= model.invoke(messages)
result

AIMessage(content='Bonjour ! Comment allez-vous ? \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 21, 'total_tokens': 33, 'completion_time': 0.021818182, 'prompt_time': 0.001321269, 'queue_time': 0.24788446200000003, 'total_time': 0.023139451}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b918c015-a3b4-4a57-a838-cf7ef0f186cf-0', usage_metadata={'input_tokens': 21, 'output_tokens': 12, 'total_tokens': 33})

In [38]:
from langchain_core.output_parsers import StrOutputParser

parser= StrOutputParser()
parser.invoke(result)

'Bonjour ! Comment allez-vous ? \n\n'

### Using LCEL(LangChain Expression Language)-->> Chain the Components

In [39]:
## This is LCEL

chain= model|parser

chain.invoke(messages)

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, ça va ?** (Informal)\n* **Coucou, comment vas-tu ?** (Very informal, used with friends and family)\n\n\nLet me know if you\'d like more options or have other phrases you want to translate!\n'

# Prompt Templates

In [40]:

from langchain_core.prompts import ChatPromptTemplate

generic_template= "Translate the following into {language}:-"

prompt= ChatPromptTemplate.from_messages(
        [
            ("system", generic_template),
            ("user","{text}")
        ]
)

In [41]:
result= prompt.invoke({"language":"French", "text":"Hello How are you?"})

In [42]:
result.to_messages()

[SystemMessage(content='Translate the following into French:-', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello How are you?', additional_kwargs={}, response_metadata={})]

## Chaining together All Three Components with LCEL


In [43]:

chain=prompt|model|parser

chain.invoke({"language":"French", "text":"Hello How are you?"})

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, ça va ?** (Informal)\n* **Coucou, comment vas-tu ?** (Very informal, used with friends or family)\n\n\nLet me know if you\'d like to know the translation for other greetings!\n'